# Tableau 
* To-Do: 
* Move function outside of project_size outside

In [1]:
def project_size(row):
    if (row.tircp > 0) and (row.tircp < p25):
        return "Small"
    elif (row.tircp > p25) and (row.tircp < p50):
        return "Medium"
    elif row.tircp > p50:
        return "Large"
    elif row.tircp == 0:
        return "$0 recorded for TIRCP"
    else:
        return "Medium"

In [ ]:
def tableau():
    # Load in cleaned project sheets
    df = A1_data_prep.clean_project()

    # Create new cols
    df = df.assign(
        Expended_Percent=(
            df["project_expended_amount"] / df["project_allocated_amount"]
        ).fillna(0),
        Allocated_Percent=(
            df["project_allocated_amount"] / df["project_tircp_award_amount__$_"]
        ).fillna(0),
        Unallocated_Amount=(
            df["project_tircp_award_amount__$_"] - df["project_allocated_amount"]
        ).fillna(0),
        Projects_Funded_Percent=(
            df["project_tircp_award_amount__$_"] / df["project_total_project_cost"]
        ).fillna(0),
    )

    # Replace districts & counties with their full names
    df["project_district"] = df["project_district"].replace(crosswalks.full_ct_district)
    df["project_county"] = df["project_county"].replace(crosswalks.full_county)

    # Categorize projects into expended % bins
    df["Expended_Percent_Group"] = df.apply(lambda x: expended_percent(x), axis=1)

    # Categorize projects whether they are ahead/behind/0 expenditures/etc
    df["Progress"] = df.apply(progress, axis=1)

    # Categorize projects whether they are large/small/med based on TIRCP amount
    df = df.rename(columns={"project_tircp_award_amount__$_": "tircp"})

    # Which projects are large,small, medium
    p75 = df.tircp.quantile(0.75).astype(float)
    p50 = df.tircp.quantile(0.50).astype(float)
    p25 = df.tircp.quantile(0.25).astype(float)

    df["Project_Category"] = df.apply(lambda x: project_size(x), axis=1)

    # Clean up column names
    df.columns = (
        df.columns.str.replace("[_]", " ")
        .str.replace("project", "")
        .str.title()
        .str.strip()
    )
    """
    # Write to GCS
    with pd.ExcelWriter(f"{GCS_FILE_PATH}Script_Tableau_Sheet.xlsx") as writer:
        df.to_excel(writer, sheet_name="Data", index=False)
    """
    return df